In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
dataset = "../../all"
data = pd.read_csv(dataset+'/training_set.csv')
list(data)
data

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1
5,615,59752.4147,1,-1061.457031,6.472994,1
6,615,59752.4224,3,-524.954590,3.552751,1
7,615,59752.4334,4,-393.480225,3.599346,1
8,615,59752.4435,5,-355.886780,10.421921,1
9,615,59767.2968,2,-548.013550,3.462291,1


In [3]:
#first make the mjd integer and if we have duplicated values take the means
#put all this is a black box train it on GP then out put to a file
#input to model object and pace
#notes : TimeSeriesSplit
#update : correct the prdiction list then add cross validation.

In [4]:
objects = data.drop_duplicates(subset=['object_id'])['object_id']
data['mjd'] = data['mjd'].astype(int)
grouped = data[data['object_id'] == 615 ]
grouped = data.groupby(['mjd','passband']).mean()
grouped.reset_index(level=['passband','mjd'],inplace=True)
grouped.mjd = grouped.mjd - grouped.mjd.min()
subset = grouped[['mjd', 'passband']]
tuples = [tuple(x) for x in subset.values]

In [14]:
def generateY(tuples,interval,maxVal):
    allMjds = list(range(0,maxVal+1,interval))
    allPassbands = [i for i in range(0,6)]
    allPredict = []
    for i in range(len(allMjds)) :
        allPredict = allPredict + [(m, p) for m in [allMjds[i]] for p in allPassbands]
    toPredict = set(allPredict) - set(tuples)
    toPredict = list(toPredict)
    toPredict.sort(key=lambda tup: tup[1])
    return pd.DataFrame(toPredict,columns=['mjd', 'passband'])

In [16]:
p = generateY(tuples,1,data.mjd.max() - data.mjd.min())
p[p['passband'] == 0 ].mjd

0       771
1       910
2         9
3       552
4       691
5       287
6       830
7       410
8       953
9        68
10      611
11      750
12      873
13      469
14     1012
15      127
16      250
17      537
18      133
19     1080
20      676
21      256
22      815
23      395
24      938
25       53
26      176
27      735
28      315
29      858
       ... 
904     397
905     940
906      55
907     598
908     178
909     721
910     317
911     860
912     440
913     999
914      98
915     524
916     360
917     663
918     786
919     925
920      24
921     583
922     163
923     706
924     302
925     845
926     425
927     968
928      83
929     222
930     765
931     345
932    1052
933     484
Name: mjd, Length: 934, dtype: int64

In [47]:
def gaussianFill(objectDf,maxVal,interval):
    grouped = objectDf.groupby(['mjd','passband']).mean()
    grouped.reset_index(level=['passband','mjd'],inplace=True)
    grouped.mjd = grouped.mjd - grouped.mjd.min()
    subset = grouped[['mjd', 'passband']]
    tuples = [tuple(x) for x in subset.values]
    predict = generateY(tuples,interval,maxVal)
    result = pd.DataFrame({'object_id':[],'mjd':[],'passband':[],'flux':[],'flux_err':[]})
    frames = []
    for i in range(0,6):
        flux = []
        filtered = grouped[grouped['passband'] == i ]
        X_train = filtered[['mjd']]
        y_train = filtered[['flux','flux_err']]
        kernel = DotProduct() + WhiteKernel()
        gaussian = GaussianProcessRegressor()
        gaussian.fit(X_train,y_train)
        predictable = pd.DataFrame(predict[predict['passband'] == 0 ].mjd,columns=['mjd'])
        flux = gaussian.predict(predictable)
        ids = [objectDf['object_id'].iloc[0]]*len(flux)
        passband = [i]*len(flux)
        df = pd.DataFrame({'object_id':ids,'passband':passband,'mjd':predictable.mjd,'flux':[i[0] for i in flux],'flux_err':[i[1] for i in flux]})
        frames = frames + [df]
    result = pd.concat(frames)
    return pd.concat(frames)

In [48]:
result = pd.DataFrame({'object_id':[],'mjd':[],'passband':[],'flux':[],'flux_err':[]})
predictions = []
for o in objects:
    predictions.append(gaussianFill(data[data['object_id'] == o ],data.mjd.max() - data.mjd.min(),1))
    break;
result = pd.concat(predictions)
result.to_csv(dataset+'/flux_predictions.csv')